# Debugging

In [2]:
import pandas as pd
import numpy as np
import yaml
import os

In [13]:
# Localise with . files 
if os.getcwd().split('\\')[-1] == 'scripts':
    config_filename = '../config_files/ds_config.yml'
else:
    config_filename = './config_files/ds_config.yml'

In [9]:
config_filename

'./config_files/ds_config.yml'

In [14]:
with open(config_filename, "r") as yml_file:
    config = yaml.safe_load(yml_file)

In [17]:
config['DS35'].keys()

dict_keys(['T_Drive_files', 'Household_Files', 'Person_Files', 'forecast_ID'])

In [18]:
config['DS35']['T_Drive_files']

{'DS35_2016': 'T:/socioec/Current_Projects/XPEF31/abm_csv/mgra13_based_input2016_01.csv',
 'DS35_2018': 'T:/socioec/Current_Projects/XPEF31/abm_csv/mgra13_based_input2018_01.csv',
 'DS35_2020': 'T:/socioec/Current_Projects/XPEF31/abm_csv/mgra13_based_input2020_01.csv',
 'DS35_2023': 'T:/socioec/Current_Projects/XPEF31/abm_csv/mgra13_based_input2023_01.csv',
 'DS35_2025': 'T:/socioec/Current_Projects/XPEF31/abm_csv/mgra13_based_input2025_01.csv',
 'DS35_2026': 'T:/socioec/Current_Projects/XPEF31/abm_csv/mgra13_based_input2026_01.csv',
 'DS35_2029': 'T:/socioec/Current_Projects/XPEF31/abm_csv/mgra13_based_input2029_01.csv',
 'DS35_2030': 'T:/socioec/Current_Projects/XPEF31/abm_csv/mgra13_based_input2030_01.csv',
 'DS35_2032': 'T:/socioec/Current_Projects/XPEF31/abm_csv/mgra13_based_input2032_01.csv',
 'DS35_2035': 'T:/socioec/Current_Projects/XPEF31/abm_csv/mgra13_based_input2035_01.csv',
 'DS35_2040': 'T:/socioec/Current_Projects/XPEF31/abm_csv/mgra13_based_input2040_01.csv',
 'DS35_204

In [12]:
config['DS35'].keys()

dict_keys(['DS35_2016', 'DS35_2018', 'DS35_2020', 'DS35_2023', 'DS35_2025', 'DS35_2026', 'DS35_2029', 'DS35_2030', 'DS35_2032', 'DS35_2035', 'DS35_2040', 'DS35_2045', 'DS35_2050', 'DS35_households_2016', 'DS35_persons_2016', 'forecast_ID'])

In [4]:
# Importing Dantes output
dante_output = pd.read_csv('C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-73 DS39 (Revised) Forecast Output QC/Data/mgra_DS42_ind_QA.csv')

In [10]:
'year' in dante_output.columns

True

In [6]:
# Importing my output 
calvin_output = pd.read_csv(r'J:/DataScience/DataQuality/QAQC/forecast_automation/mgra_series_13_outputs_CSV_data/aggregated_data/mgra_DS42_ind_QA.csv')

c:\Users\cra\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3552: DtypeWarning: Columns (112,113) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Compare

In [13]:
small_dante_output = dante_output[['year', 'mgra', 'emp_total']]
small_dante_output

,year,mgra,emp_total
0,2016,1,10
1,2016,2,1
2,2016,3,5
3,2016,4,1
4,2016,5,0
...,...,...,...
299021,2050,22998,22
299022,2050,22999,147
299023,2050,23000,2
299024,2050,23001,7


In [14]:
small_calvin_output = calvin_output[['year', 'mgra', 'emp_total']]
small_calvin_output

,year,mgra,emp_total
0,2016,1,10
1,2016,2,1
2,2016,3,5
3,2016,4,1
4,2016,5,0
...,...,...,...
299021,2050,22998,22
299022,2050,22999,147
299023,2050,23000,2
299024,2050,23001,7


In [15]:
small_dante_output['emp_total'].sum()

23920073

In [16]:
small_calvin_output['emp_total'].sum()

23920035

In [18]:
compare_emp_total = small_dante_output.merge(small_calvin_output, how='left', on=['year', 'mgra'], suffixes=['_Dante', '_Calvin'])
compare_emp_total

,year,mgra,emp_total_Dante,emp_total_Calvin
0,2016,1,10,10
1,2016,2,1,1
2,2016,3,5,5
3,2016,4,1,1
4,2016,5,0,0
...,...,...,...,...
299021,2050,22998,22,22
299022,2050,22999,147,147
299023,2050,23000,2,2
299024,2050,23001,7,7


In [19]:
compare_emp_total['Diff'] = compare_emp_total['emp_total_Dante'] - compare_emp_total['emp_total_Calvin']

In [22]:
non_zero_diff = compare_emp_total[compare_emp_total['Diff'] != 0]
non_zero_diff

,year,mgra,emp_total_Dante,emp_total_Calvin,Diff
23005,2018,4,0,1,-1
23064,2018,63,4,3,1
23092,2018,91,141,140,1
23099,2018,98,4,3,1
23105,2018,104,1,0,1
...,...,...,...,...,...
298979,2050,22956,2836,2846,-10
298980,2050,22957,87,86,1
298981,2050,22958,97,98,-1
298987,2050,22964,65,68,-3


In [28]:
non_zero_diff = non_zero_diff.sort_values('Diff', ascending=False)
non_zero_diff

,year,mgra,emp_total_Dante,emp_total_Calvin,Diff
218900,2035,11883,7332,7301,31
165197,2030,4184,1371,1351,20
165183,2030,4170,11118,11100,18
229924,2035,22907,1259,1241,18
142181,2029,4170,10676,10659,17
...,...,...,...,...,...
215530,2035,8513,903,919,-16
165185,2030,4172,7021,7039,-18
143794,2029,5783,10760,10778,-18
166796,2030,5783,10883,10903,-20


In [29]:
non_zero_diff.to_csv('C:/Users/cra/Desktop/Dante_Calvin_Diff.csv')

In [25]:
len(set(non_zero_diff['mgra']))

5524